# 2차 라우터

In [ ]:
from typing import Dict, List, Optional, Literal
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from state import State

from db import DEPARTMENTS


# 부서 정보 dict (이메일과 슬랙 채널)
DEPARTMENTS = {
    "재무": {"name": "재무", "email": "fi@gaida.play.com", "slack": "#ask-fi"},
    "총무": {"name": "총무", "email": "ga@gaida.play.com", "slack": "#ask-ga"},
    "인프라": {"name": "인프라", "email": "in@gaida.play.com", "slack": "#ask-in"},
    "보안": {"name": "보안", "email": "se@gaida.play.com", "slack": "#ask-se"},
    "인사": {"name": "인사", "email": "hr@gaida.play.com", "slack": "#ask-hr"},
}

class SecondaryRouter:
    """2차 라우터: LLM 기반 RAG vs 담당자 안내 분류 시스템"""
    
    def __init__(self, llm_model: ChatOpenAI):
        self.llm = llm_model

    def classify_with_llm(self, question: str) -> Dict[str, str]:    # question: str 이 어떤 state인지?
        """
        LLM을 사용한 통합 분류: RAG vs 담당자 안내 + 부서 결정
        """
        system_prompt = """
        당신은 사내(gaida play studio) HR 챗봇의 질문 분류 전문가입니다.
        사용자의 질문을 분석하여 어떻게 처리할지 결정해주세요.

        # 분류 기준

        ## 1. RAG 처리 대상 (route: "rag")
        - 회사 내부 규정, 정책, 제도에 대한 일반적인 질문
        - 문서에서 답변을 찾을 수 있는 정보성 질문
        - 예시:
        * "연차 규정이 어떻게 되나요?"
        * "재택근무 정책을 알려주세요"
        * "복지제도에는 무엇이 있나요?"
        * "근무시간은 어떻게 되나요?"
        * "휴가 신청 방법을 알려주세요"
        * "장비 사용 규칙이 궁금해요"

        ## 2. 담당자 안내 대상 (route: "department")
        - 개인별 맞춤 처리가 필요한 질문
        - 실시간 처리나 승인이 필요한 업무
        - 문제 해결이나 신고가 필요한 상황
        - 개별 상담이 필요한 민감한 사안

        ### 부서별 담당 업무:
        - **재무**: 급여, 세금, 예산, 회계, 지출, 송금, 계산서, 청구서, 지급, 비용, 환급
        - **총무**: 사무실, 비품, 물품, 구매, 수령, 우편, 사무용품, 시설, 행사, 차량, 청소, 자산, 출장, 숙박, 교통
        - **인프라**: 서버, 네트워크, 컴퓨터, IT, 소프트웨어, 장비, 시스템, 접속, VPN, 계정, 접근
        - **보안**: 보안, 해킹, 정보, 유출, 침해, 랜섬웨어, 백신, 데이터, 비밀번호, 방화벽, 악성코드, 암호
        - **인사**: 개별 급여 문의, 채용, 인사평가, 퇴직, 입사, 퇴사, 평가, 승진, 개인적 근무 상담, 직장 내 괴롭힘 신고

        # 판단 원칙
        - 일반적인 제도/정책 문의 → RAG
        - 개인별/개별적 처리가 필요한 문의 → 담당자 안내
        - 실시간 문제 해결이 필요한 경우 → 담당자 안내
        - 민감한 개인 정보 관련 → 담당자 안내
        
        # 응답 형식
        다음 JSON 형식으로만 응답해주세요:

        RAG 처리인 경우:
        {"route": "rag"}

        담당자 안내인 경우:
        {"route": "department", "department": "부서명"}

        부서명은 반드시 다음 중 하나여야 합니다: 재무, 총무, 인프라, 보안, 인사

        부득이하게 재무, 총무, 인프라, 보안 부서에 해당하지 않을 경우에는 인사로 지정해주세요.
                """
                
        user_prompt = f"""
        사용자 질문: "{question}"

        위 질문을 분석하여 RAG 처리할지, 담당자 안내할지 결정하고 JSON 형식으로 응답해주세요.
                """

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        
        try:
            response = self.llm.invoke(messages)
            response_text = response.content.strip()
            
            # JSON 파싱 시도
            import json
            try:
                result = json.loads(response_text)
                return result
            except json.JSONDecodeError:
                # JSON 파싱 실패시 텍스트에서 추출
                if "rag" in response_text:
                    return {"route": "rag"}
                else:
                    # 부서명 추출 시도
                    for dept in DEPARTMENTS.keys():
                        if dept in response_text:
                            return {"route": "department", "department": dept}
                    return {"route": "department", "department": "인사"}
                
        except Exception as e:
            print(f"LLM 분류 오류: {e}")
            # 기본값: 인사팀 담당자 안내로 라우팅
            return {"route": "department", "department": "인사"}

    def route_question(self, state: State) -> State:
        """
        LLM 기반 질문 분류 및 라우팅
        """
        question = state['refined_question']
        
        print(f" LLM 기반 질문 분류 시작...")
        
        # LLM을 통한 통합 분류
        classification_result = self.classify_with_llm(question)
        
        route = classification_result.get("route")
        department = classification_result.get("department")
        
        print(f" 분류 결과: {classification_result}")
        
        if route == "rag":
            # RAG 처리로 분류
            state['is_rag_suitable'] = True 
            state['department_info'] = None
            state['answer_type'] = "rag_answer"  # RAG 처리 상태
            print("➡️ RAG 시스템으로 라우팅")
        else:
            # 담당자 안내로 분류
            state['is_rag_suitable'] = False 
            state['department_info'] = DEPARTMENTS.get(department, DEPARTMENTS["인사"])
            state['answer_type'] = "department_contact"
            print(f"➡️ {department}팀 담당자 안내로 라우팅")
        
        return state

    def generate_department_response(self, state: State) -> str:
        """
        담당자 안내 응답 생성
        """
        department = state.get('department_info') 

        if not department:
            return "해당 문의사항은 인사팀으로 문의하시면 정확하고 빠른 답변을 받으실 수 있습니다.\n\n(hr@gaida.play.com / #ask-hr)\n\n추가 질문이 있으시면 언제든 말씀해 주세요! 😊"
        
        response = f"""
        해당 문의사항은 **{department['name']}팀**으로 문의하시면 정확하고 빠른 답변을 받으실 수 있습니다.

        📧 **이메일**: {department['email']}
        💬 **슬랙**: {department['slack']}
        
        추가 질문이 있으시면 언제든 말씀해 주세요! 😊
        """
        
        return response.strip()

    def process_secondary_routing(self, state: State) -> State:
        """
        2차 라우터 전체 처리 프로세스
        """
        try:
            # 1. 질문 분류 및 라우팅
            state = self.route_question(state)
            
            # 2. 각 경로별 완전 처리
            if state['answer_type'] == "department_contact":
                state['final_answer'] = self.generate_department_response(state)
            elif state['answer_type'] == "rag_answer":
                # RAG 처리 대기 상태 설정
                state['final_answer'] = ""  # RAG 모듈에서 채울 예정
                
        except Exception as e:
            print(f"2차 라우터 처리 중 오류: {e}")
            # 에러 시 기본 처리
            state['answer_type'] = "department_contact"
            state['department_info'] = DEPARTMENTS.get("인사", DEPARTMENTS["인사"])
            state['final_answer'] = self.generate_department_response(state)
            
        return state



# 테스트 코드
def test_secondary_router():
    """2차 라우터 테스트"""
    
    # LLM 모델 초기화
    try:
        llm = ChatOpenAI(model='gpt-4o', temperature=0)
        router = SecondaryRouter(llm)
    except:
        print("⚠️ LLM 모델 초기화 실패. API 키를 확인해주세요.")
        return
    
    # 테스트 케이스들
    test_cases = [
        # RAG 처리 예상 질문들
        "근무 스케쥴을 어떻게 바꿀수 있나요?",
        "재택근무 정책이 궁금해요",
        "복지제도에는 뭐가 있나요?",
        "근무시간은 어떻게 되나요?",
        "휴가 신청 방법을 알려주세요.",
        "다태아의 경우 출산 휴가 규정이 어떻게 되나요?",
        
        # 담당자 안내 예상 질문들  
        "이번 달 급여명세서를 확인하고 싶어요",  # 재무
        "노트북이 고장났는데 교체해주세요",     # 인프라
        "출장비 정산을 신청하고 싶습니다",       # 총무
        "보안 사고가 발생했어요",              # 보안
        "인사평가 결과를 문의하고 싶습니다",     # 인사
        "회계처리 관련 문의가 있습니다",        # 재무
        "사무용품을 주문하고 싶어요",           # 총무
        "VPN 접속이 안 돼요",                # 인프라
        "비밀번호가 유출된 것 같아요",          # 보안
        "퇴사를 하고 싶어요.",
        "직장내 괴롭힘을 신고하고 싶어요",
    ]
    
    print("=" * 80)
    print("HR 챗봇 2차 라우터 테스트 (LLM 기반 분류)")
    print("=" * 80)
    
    for i, question in enumerate(test_cases, 1):
        print(f"\n[테스트 {i}] 질문: {question}")
        print("-" * 60)
        
        # 상태 초기화
        state = State(
            user_question=question,
            refined_question=question,     # 1차 전처리에서 정제된 질문
            is_hr_question=True,           # 1차 라우터에서 HR 질문으로 판정됨
            is_rag_suitable=False,         # 2차 라우터에서 결정
            department_info=None,          # 2차 라우터에서 결정  
            retrieved_docs=[],             # RAG 처리시 사용
            answer_type="pending",         # 초기 상태
            final_answer=""                # 최종 답변
        )
        
        # 2차 라우터 처리
        result_state = router.process_secondary_routing(state)
        
        # 결과 출력
        if result_state['is_rag_suitable']: 
            print("🔍 ➡️ RAG 시스템으로 전달")
            print(f"📋 답변 유형: {result_state['answer_type']}")
        else:
            print(f"👥 ➡️ {result_state['department_info']['name']}팀 담당자 안내") 
            print(f"📋 답변 유형: {result_state['answer_type']}")
            print(f"\n💬 응답:\n{result_state['final_answer']}")

if __name__ == "__main__":
    test_secondary_router()